# Text generation using a RNN with eager execution

(Adapted from the tensorflow example https://www.tensorflow.org/tutorials/sequences/text_generation)

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Goals

- Understand the evolution of samples during training, and the relationship between the number of training ephocs and the quality of the generated output. 
- Produce (or download) your own textual corpus to experiment with parameters, training, and results. In [his article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) cited above, Karpathy plays with sources including Shakespeare, Wikipedia articles, Algebraic Geometry (Latex), and Linux source code, and baby names.
E

## Setup

### Import TensorFlow and other libraries

In [45]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [46]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# path_to_file = "try_with_your_own_file.txt"




### Read the data

First, look in the text.

In [47]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 747567 characters


In [48]:
# Take a look at the first 250 characters in text
print(text[:250])



Reading Reference




Narratology: Introduction to the theory of Narrative, 3rd ed. Mieke Bal. 2009.

A narrative test is a text in which an agent or subject conveys to an addressee ('tells' the reader) a story in a particular medium, such as langu


In [49]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

110 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [50]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [51]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  '\x0c':   2,
  '\r':   3,
  ' ' :   4,
  '!' :   5,
  '"' :   6,
  '#' :   7,
  '%' :   8,
  '&' :   9,
  "'" :  10,
  '(' :  11,
  ')' :  12,
  '*' :  13,
  '+' :  14,
  ',' :  15,
  '-' :  16,
  '.' :  17,
  '/' :  18,
  '0' :  19,
  ...
}


In [52]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\nReading Ref' ---- characters mapped to int ---- > [ 1  1 52 69 65 68 73 78 71  4 52 69 70]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. 

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [53]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])





R
e
a


The `batch` method lets us easily convert these individual characters to sequences of the desired size. 

In [54]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\n\nReading Reference\n\n\n\n\nNarratology: Introduction to the theory of Narrative, 3rd ed. Mieke Bal. 2009'
".\n\nA narrative test is a text in which an agent or subject conveys to an addressee ('tells' the reade"
'r) a story in a particular medium, such as language, imager, sound, buildings, r a combination thereo'
"f. A story is the fontent of that text, and produces a particular manifestation, inflection, and 'col"
"oring' of a fabula; the fabula is presented in a certain manner. A fabula is a series of logically an"


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [55]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [56]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\nReading Reference\n\n\n\n\nNarratology: Introduction to the theory of Narrative, 3rd ed. Mieke Bal. 200'
Target data: '\nReading Reference\n\n\n\n\nNarratology: Introduction to the theory of Narrative, 3rd ed. Mieke Bal. 2009'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [57]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    1
  input: 1 ('\n')
  expected output: 52 ('R')
Step    2
  input: 52 ('R')
  expected output: 69 ('e')
Step    3
  input: 69 ('e')
  expected output: 65 ('a')
Step    4
  input: 65 ('a')
  expected output: 68 ('d')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [58]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [59]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

Use `CuDNNGRU` if running on GPU.  

In [60]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [61]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [62]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/sequences/images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [63]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 110) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length: 

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           28160     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 110)           112750    
Total params: 4,079,214
Trainable params: 4,079,214
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. 

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [65]:
sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) 
#sample_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [66]:
sampled_indices

array([ 41,  77,  62,  51,  23,   3,  20,   8,  52,   8,  74,   8,  14,
       107,  12,   9,   9,  64,  21,   7, 108,  33,  94,  25,  94,  50,
        89,  21,  86, 105,  84,  80,  82,  81,  41,  73,  86,  68,  58,
        17,  85,  64,  95,  54,  61,  43,  75,  28,  22,  80,   1,  12,
        75,  79,  68,  46,  57, 104,  89,  77,  96,  81,   1,  69,  28,
        55,  74, 103,  74,  18,  12,  38,  90,  94,   1,  78,  94,  92,
        38,  60,  58,  60,  62,  65,  98,  60,  45,  52,  61,   3,   9,
        22,  14,  44,  97,  28,  99,  55,   9,  73])

Decode these to see the text predicted by this untrained model:

In [67]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'pologist Claude Levi-Strauss characterizes two modes of thought, the mythical and the scientific. My'

Next Char Predictions: 
 'Gm]Q4\r1%R%j%+Ó)&&`2#Õ>\x8d6\x8dPy2vÑtprqGivdX.u`\x8eT[Ik93p\n)kodLWÐym\x93q\ne9UjÉj/)Dz\x8d\nn\x8d}DZXZ]a\x9dZKR[\r&3+J\x999\x9fU&i'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions. 

Because our model returns logits, we need to set the `from_logits` flag. 


In [68]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 110)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.701376


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function. 

In [69]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [70]:
# Directory where the checkpoints will be saved
checkpoint_dir = './text100'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [71]:
EPOCHS=3

In [72]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/10
116/116 [==============================] - 7s 63ms/step - loss: 2.8899
Epoch 2/10
116/116 [==============================] - 7s 57ms/step - loss: 2.1782
Epoch 3/10
116/116 [==============================] - 6s 55ms/step - loss: 1.8924
Epoch 4/10
116/116 [==============================] - 7s 56ms/step - loss: 1.6824
Epoch 5/10
116/116 [==============================] - 6s 55ms/step - loss: 1.5467
Epoch 6/10
116/116 [==============================] - 6s 55ms/step - loss: 1.4590
Epoch 7/10
116/116 [==============================] - 6s 55ms/step - loss: 1.3957
Epoch 8/10
116/116 [==============================] - 7s 57ms/step - loss: 1.3442
Epoch 9/10
116/116 [==============================] - 7s 56ms/step - loss: 1.3018
Epoch 10/10
116/116 [==============================] - 6s 54ms/step - loss: 1.2615


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [73]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text100/ckpt_10'

In [74]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [75]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            28160     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 110)            112750    
Total params: 4,079,214
Trainable params: 4,079,214
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [76]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [77]:
# print(generate_text(model, start_string=u"ROMEO"))
print(generate_text(model, start_string=u"writing"))


writing age of poputian out of his task in the discurby identify for meanmanist. 

Most torammers in a dialogue identifying the writers. p 64.

Kear; I am working within a gunstitute act an extend works and confusianity; the term "white against the magically unist as it is playing a view of the lose is in the analogical landscape recipe it we've beel-systematically. p 135.

Ws model is space. p 138.

mayzey until the knowledge, decisions are - then sometimes surface between cembariail beautiful basic voilctives initmed. Untried At He was does long and "that is technology as jung speaks of Style class down soon a "piece?'4.

Acting. 

Thuman Popul and Gant, Derri, ose] means it to produce that rendering the to it the unrejected identician--shmists, fall people whole, observes."  The Enthore in Rode BrD5 Pronsever, those strictly advertising the piring of return to the body has my top macy and ruseual modern phase stores, or those concression, oppositioned place universe into others a 


The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output. 

We will use `tf.GradientTape` to track the gradiends. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [79]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [80]:
optimizer = tf.train.AdamOptimizer()

In [ ]:
# Training step
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions = model(inp)
              loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)
              
          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))

          if batch_n % 100 == 0:
              template = 'Epoch {} Batch {} Loss {:.4f}'
              print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efed34d30>, <tf.Tensor: id=883691, shape=(256, 3072), dtype=float32, numpy=
array([[-1.6645500e-06, -2.2062575e-06,  1.7071508e-06, ...,
         9.0128751e-06, -5.9112317e-06,  2.6318590e-05],
       [-1.6155893e-06, -1.9279382e-06,  1.1504717e-06, ...,
        -1.3546650e-05, -1.9089233e-05,  1.0909354e-06],
       [ 3.5631720e-06, -2.2063753e-06,  6.2005811e-06, ...,
        -3.1541535e-05, -8.6429291e-06, -2.2900442e-06],
       ...,
       [ 2.4355722e-06,  4.9641960e-07,  5.9189250e-07, ...,
        -3.0895822e-06, -8.0293930e-06,  1.2242006e-05],
       [-1.4867965e-06, -6.2188144e-07,  1.0489189e-06, ...,
         1.3251835e-05,  3.2291862e-07,  4.7271287e-06],
       [ 4.4425838e-07, -3.6718487e-07,  5.5877337e-07, ...,
         2.2209824e-05,  3.8150213e-07,  3.6547201e-05]], dtype=float32)>, <tf.Tensor: id=883692, shape=(1024, 3072), dtype=float32, numpy=
array([[-7.7868316e-07, -3.8747717e-06,  8.1323509e-07, ..

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdafce80>, <tf.Tensor: id=885136, shape=(256, 3072), dtype=float32, numpy=
array([[-2.08622782e-06, -1.93767005e-06,  1.45472075e-06, ...,
         2.16406279e-05, -2.43745740e-06, -1.09734192e-05],
       [ 8.34993443e-07,  2.04917114e-07, -7.41905879e-08, ...,
         6.37814310e-06,  1.38452160e-05,  1.49512971e-05],
       [-2.69676065e-07,  1.44108867e-07,  1.37633066e-07, ...,
        -4.49148647e-05, -3.05456433e-06, -2.84641392e-05],
       ...,
       [-1.95043057e-07,  1.04220426e-07,  1.13053865e-07, ...,
        -6.27214877e-06,  2.67478094e-06, -1.41209910e-06],
       [ 1.60297731e-07,  3.58165906e-08, -1.68633196e-08, ...,
        -3.73492076e-05, -7.98025940e-06,  1.39143340e-05],
       [ 7.82574489e-07,  3.95758235e-07, -1.97070321e-07, ...,
        -2.19728167e-06, -1.05059135e-05,  4.58074010e-06]], dtype=float32)>, <tf.Tensor: id=885137, shape=(1024, 3072), dtype=float32, numpy=
array([[-3.6008355e-06

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdafcb00>, <tf.Tensor: id=886580, shape=(256, 3072), dtype=float32, numpy=
array([[ 3.1036856e-07, -1.0393370e-06, -2.8686870e-07, ...,
         1.1283505e-05,  1.4970736e-05,  7.1279144e-05],
       [ 7.1035407e-07,  3.1994836e-07, -1.9404276e-07, ...,
        -1.4029039e-06, -7.5122666e-06,  1.1055386e-05],
       [ 2.0456642e-07, -1.1740746e-07, -6.9135076e-08, ...,
         9.5458654e-06,  7.1952581e-06,  5.5944576e-05],
       ...,
       [-2.5341063e-07, -1.0001706e-08,  9.6723866e-08, ...,
         4.9310693e-06, -8.1125590e-06,  2.2366833e-06],
       [ 8.2215202e-08,  6.5079938e-08, -3.6493366e-09, ...,
        -3.0755828e-06,  3.7854047e-06, -5.0751805e-05],
       [-5.9992453e-07, -1.3791150e-07,  1.1436682e-07, ...,
         2.6498366e-05, -2.1198946e-05, -1.5837681e-05]], dtype=float32)>, <tf.Tensor: id=886581, shape=(1024, 3072), dtype=float32, numpy=
array([[ 7.6546394e-07, -2.9352389e-06,  6.2912668e-06, ..

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdae76a0>, <tf.Tensor: id=888024, shape=(256, 3072), dtype=float32, numpy=
array([[-1.2291123e-07, -3.8444131e-07,  8.2069482e-07, ...,
        -3.1301683e-05, -1.8692679e-05, -8.4705671e-06],
       [-3.4729788e-07, -2.3518345e-07,  3.3864137e-07, ...,
         6.9324210e-06, -2.5442589e-05, -3.0589858e-06],
       [-5.4353217e-07, -1.3732523e-07,  8.7531467e-09, ...,
        -2.1930255e-05,  1.4878005e-05, -7.5094513e-06],
       ...,
       [ 1.4108635e-07,  1.3637670e-07,  1.7452886e-07, ...,
         7.1916552e-06, -2.4998149e-06,  3.4614297e-07],
       [ 3.2025220e-08,  1.7245545e-08, -2.3123263e-08, ...,
        -4.5908895e-05,  1.8242066e-05, -4.1493884e-05],
       [-5.1885144e-07, -1.7108722e-07,  1.2070984e-07, ...,
        -2.5062420e-05,  1.0913977e-05,  4.2917407e-05]], dtype=float32)>, <tf.Tensor: id=888025, shape=(1024, 3072), dtype=float32, numpy=
array([[-2.56345174e-06, -6.93257152e-06, -1.29555531e-06,

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdafc518>, <tf.Tensor: id=889468, shape=(256, 3072), dtype=float32, numpy=
array([[ 2.12514942e-06, -4.30143075e-07, -5.52845677e-07, ...,
        -1.90090341e-05, -4.54943984e-06,  2.65667186e-05],
       [-8.07125389e-08, -2.98955136e-08, -1.38342102e-07, ...,
         4.41361590e-06,  4.56988637e-05, -5.50793175e-06],
       [-6.61572869e-07,  3.82869132e-08,  1.87039817e-07, ...,
        -2.17803317e-05,  2.97595834e-05, -1.71621887e-05],
       ...,
       [ 5.35297545e-07,  1.91635330e-08, -9.75905508e-08, ...,
         8.15444673e-06, -4.97278643e-06,  3.25660039e-06],
       [-2.12612132e-08,  4.91199721e-08,  7.69868080e-09, ...,
        -9.18863298e-05,  5.65555201e-05,  1.87585056e-06],
       [ 1.30558050e-07,  1.16622076e-07,  3.21251832e-07, ...,
        -1.73448898e-05, -3.82582075e-05, -4.30033697e-06]], dtype=float32)>, <tf.Tensor: id=889469, shape=(1024, 3072), dtype=float32, numpy=
array([[-2.9881985e-06

...

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdafc860>, <tf.Tensor: id=914016, shape=(256, 3072), dtype=float32, numpy=
array([[ 2.50244557e-06, -2.02146134e-06, -2.58576756e-06, ...,
         1.27404355e-05, -4.48473074e-05, -1.52936973e-05],
       [-5.67024699e-07,  3.71498032e-07,  2.12241016e-08, ...,
        -4.47506827e-05, -1.86877571e-06, -2.23127226e-05],
       [ 1.83991318e-07,  1.04574873e-07, -1.19513913e-07, ...,
         6.08786431e-05, -1.78010559e-05, -1.77620186e-05],
       ...,
       [-3.91859516e-07, -4.39847128e-07, -1.71540435e-07, ...,
        -6.29241413e-07, -8.76280774e-06, -1.20971790e-05],
       [-3.44813458e-08, -6.53901893e-08, -2.58598103e-08, ...,
        -3.37648635e-05,  8.38114574e-05,  7.47408849e-05],
       [ 7.07834602e-07,  9.88336069e-07, -9.27861521e-08, ...,
         4.97772044e-06, -4.35799775e-05,  3.61911880e-05]], dtype=float32)>, <tf.Tensor: id=914017, shape=(1024, 3072), dtype=float32, numpy=
array([[ 1.2933432e-05

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdb0a320>, <tf.Tensor: id=915460, shape=(256, 3072), dtype=float32, numpy=
array([[ 3.9344209e-06, -4.9483532e-07, -2.4209007e-06, ...,
        -7.7528552e-05,  6.1206578e-05,  1.2646603e-05],
       [ 5.7941980e-07,  5.1684691e-07, -4.9253430e-08, ...,
         1.3666038e-05, -2.2019087e-05,  5.7831044e-06],
       [-5.1707451e-07, -1.4204726e-07,  1.4884253e-07, ...,
         9.1855909e-06,  4.6392259e-05,  7.1696568e-06],
       ...,
       [ 2.0476911e-07, -3.7411226e-08,  8.3456064e-07, ...,
        -1.7432087e-05,  2.8624647e-05,  9.7748889e-06],
       [ 2.4488115e-08, -9.1220777e-08,  1.5327403e-08, ...,
         2.0874262e-05, -3.5277815e-05,  8.5237498e-06],
       [-1.9883289e-06,  2.0096505e-07,  1.2107103e-06, ...,
         3.1741380e-05, -3.8089918e-05, -5.7156889e-05]], dtype=float32)>, <tf.Tensor: id=915461, shape=(1024, 3072), dtype=float32, numpy=
array([[ 1.6539412e-05, -6.1931962e-07,  8.4177209e-06, ..

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdafc9e8>, <tf.Tensor: id=916904, shape=(256, 3072), dtype=float32, numpy=
array([[-6.9358712e-06, -4.4834796e-06,  1.6087159e-06, ...,
        -9.5508958e-06, -4.6902609e-05, -5.0258775e-05],
       [-6.2540948e-07, -3.1282300e-07,  2.9097089e-07, ...,
        -1.1741785e-05, -1.7875271e-06,  3.1817744e-05],
       [-9.8166913e-07, -3.9934869e-07, -9.3014691e-08, ...,
         1.7001526e-05, -2.1415077e-05, -3.4060045e-05],
       ...,
       [ 1.4393236e-06, -8.5470468e-08, -2.5556642e-07, ...,
        -6.1212204e-06, -2.4985102e-05, -2.8062135e-05],
       [-2.5305718e-08,  4.2107306e-08,  9.7515702e-08, ...,
         4.2639786e-05, -1.7833217e-05,  3.2730462e-05],
       [-1.4031278e-06, -5.3073927e-07,  1.5729166e-06, ...,
        -1.0002765e-04,  8.5415508e-05,  2.0788517e-05]], dtype=float32)>, <tf.Tensor: id=916905, shape=(1024, 3072), dtype=float32, numpy=
array([[-1.9646044e-05, -2.8900893e-05, -2.6858557e-05, ..

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdafc0f0>, <tf.Tensor: id=918348, shape=(256, 3072), dtype=float32, numpy=
array([[ 2.0570719e-06, -1.1953455e-07, -3.3576157e-06, ...,
        -6.2512503e-05, -8.3862933e-06,  3.0087789e-05],
       [ 4.5852175e-08,  7.5875391e-08,  2.3280101e-07, ...,
        -7.0192950e-06, -2.6150069e-06, -8.2028528e-06],
       [-1.0099081e-07,  7.8367020e-09, -8.5380947e-08, ...,
        -4.5354373e-06,  2.2442557e-05, -7.5808834e-06],
       ...,
       [ 2.3055125e-06,  7.7460390e-07, -1.4397632e-07, ...,
        -7.1472514e-06,  2.7451331e-05,  1.1520820e-05],
       [ 6.9272237e-09,  1.7133446e-07,  3.3843257e-08, ...,
        -3.8881037e-05, -1.1255582e-05, -3.0078523e-05],
       [-5.1916669e-07,  2.4943844e-07,  5.7096432e-08, ...,
        -3.3956308e-05,  2.2144081e-05,  3.5611756e-05]], dtype=float32)>, <tf.Tensor: id=918349, shape=(1024, 3072), dtype=float32, numpy=
array([[ 2.2079212e-05,  2.7550195e-06, -2.6931521e-06, ..

[<tensorflow.python.framework.ops.IndexedSlices object at 0x7f4efdb0af60>, <tf.Tensor: id=919792, shape=(256, 3072), dtype=float32, numpy=
array([[-3.71483884e-06, -3.15255147e-06,  2.62256322e-06, ...,
         1.59173069e-05, -1.92328407e-05, -2.98543237e-05],
       [-1.20727543e-06, -3.34444508e-07,  3.48126008e-07, ...,
         6.47762936e-05, -8.63289461e-06, -1.20562345e-05],
       [ 3.14985670e-07, -3.92516739e-08,  7.32983141e-09, ...,
        -4.67955470e-05,  1.08446684e-05, -1.33166641e-05],
       ...,
       [ 1.89275147e-06,  1.19670881e-06, -1.00265663e-06, ...,
        -6.16513398e-06, -1.40243974e-05, -1.59270458e-05],
       [-6.27527825e-08,  8.70638033e-08,  2.89620736e-08, ...,
         9.38427888e-07,  2.15831624e-05, -4.53977409e-05],
       [ 6.41095141e-07,  1.71443173e-06,  5.65115066e-08, ...,
        -9.20488164e-05,  1.44293663e-04,  3.86730935e-05]], dtype=float32)>, <tf.Tensor: id=919793, shape=(1024, 3072), dtype=float32, numpy=
array([[-2.95812533e-0